LZ78
* The dictionary consist of phrases numbered from 0 upwards:<br>
D = {Z0, Z1, Z2, ...}

* Each new phrase inserted to the dictionary gets the next free number.

* Initialy, the only phrase is the empty string Z0 = ''

* Suppose we have so far computed the parsing T1...T(j-1) for T[0...i) and the next phrase Tj starts at position i.<br>
Let Zk in D ne the longest phrase in the dictionary that is a prefix of T[i...n-1).<br>
The next phrase is Tj = Zj = T[i...i+|Zk|] = Zk*t(i+|Zk|), and it is inserted to the dictionary.

* The phrase Tj is encoded as a pair (k, t(i+|Zk|))

In [1]:
from math import log2,ceil
from tabulate import tabulate

In [2]:
def lz78_encode(T):
    """
    lz78 encodeing.
    Input: text T to encode
    Output: a table (list of lists) with the next columns:
            - index of dictionary (not the python dictionary (not the python object))
            - Phrase, the partition of text T
            - Encoding, tuple like (index, symbol)
            - number of bits, that takes to represent the codeword
    """
    ALPHABET        = set(list(T))
    bits_per_symbol = ceil(log2(len(ALPHABET)))
    D               = {'':0}

    #        index  |  phrase  | Encoding  | # of bits
    table = [[D[''] , ''       , None      , None     ]]

    i = 0
    n = len(T)
    while i < n:
        # find the max prefix of text T that exist in dictionary D
        if T and T[:i] in D and T[:i+1] not in D:
            # update table
            table.append([len(D),\
                        T[:i+1],\
                        (D[T[:i]], T[i]),\
                        "{}+{}".format(ceil(log2(len(D))), bits_per_symbol)])
            
            # update dictionary
            D[table[-1][1]] = table[-1][0]
            
            # sliding window moving
            T = T[i+1:]
            i = 0
        else:
            i += 1
    
    return table

In [3]:
def lz78_decode(tokens, bits_per_symbol):
    """
    lz78 decoding.
    Input: tokens and number of bits per symbol.
    Output: a table (list of lists) with the next columns:
            - index of dictionary (not the python dictionary (not the python object))
            - Phrase, the decoded text
            - Encoding (the tokens)
            - number of bits, that takes to represent the codeword
    Note: this table must be the same as in lz78_encode!
    """
    D = {'':0}

    #        index  |  phrase  | Encoding  | # of bits
    table = [[D[''] , ''       , None      , None     ]]

    for index, symbol in tokens:
        # update table
        table.append([len(D),\
                    table[index][1] + symbol,\
                    (index, symbol),\
                    "{}+{}".format(ceil(log2(len(D))), bits_per_symbol)])
            
        # update dictionary
        D[table[-1][1]] = table[-1][0]
    
    return table

In [4]:
# example for PDF 8 slide 7:
T = "badadadabaab"
ALPHABET        = set(list(T))
bits_per_symbol = ceil(log2(len(ALPHABET)))
print("Text: {}, ALPHABET: {}".format(T, ALPHABET))

# encode
table = lz78_encode(T)
print(tabulate(table, headers=["index", "Phrase", "Encoding", "# of bits"], tablefmt="pretty"))
tokens = [row[2] for row in table][1:] # remove the first None
print("Encoding: {}".format(tokens))

# decode 
table = lz78_decode(tokens, bits_per_symbol)
print(tabulate(table, headers=["index", "Phrase", "Encoding", "# of bits"], tablefmt="pretty"))
phrase = [row[1] for row in table][1:] # remove the first ''
print("Decoding: {}".format(''.join(phrase)))

Text: badadadabaab, ALPHABET: {'a', 'd', 'b'}
+-------+--------+----------+-----------+
| index | Phrase | Encoding | # of bits |
+-------+--------+----------+-----------+
|   0   |        |          |           |
|   1   |   b    | (0, 'b') |    0+2    |
|   2   |   a    | (0, 'a') |    1+2    |
|   3   |   d    | (0, 'd') |    2+2    |
|   4   |   ad   | (2, 'd') |    2+2    |
|   5   |  ada   | (4, 'a') |    3+2    |
|   6   |   ba   | (1, 'a') |    3+2    |
|   7   |   ab   | (2, 'b') |    3+2    |
+-------+--------+----------+-----------+
Encoding: [(0, 'b'), (0, 'a'), (0, 'd'), (2, 'd'), (4, 'a'), (1, 'a'), (2, 'b')]
+-------+--------+----------+-----------+
| index | Phrase | Encoding | # of bits |
+-------+--------+----------+-----------+
|   0   |        |          |           |
|   1   |   b    | (0, 'b') |    0+2    |
|   2   |   a    | (0, 'a') |    1+2    |
|   3   |   d    | (0, 'd') |    2+2    |
|   4   |   ad   | (2, 'd') |    2+2    |
|   5   |  ada   | (4, 'a') |    

LZW Terry Welch (1984)

In [5]:
def lzw_encode(T):
    """
    lzw encoder
    Input: text T to encode
    Output: code (list) and table represent the dictionary (tabulate)
    """
    ALPHABET = sorted(list(set(list(T)))) # sort for strinct order..
    D        = {} # dictionary {phrase:code}
    E        = [] # encoding. list of tokens
    table    = [] # presenting the full dictionary D as table (tabulate type)
    
    EOF      = '$'
    T        += EOF # assume '$' doesnt in T

    # Dictionary <- single characters
    for i in range(len(ALPHABET)):
        D[ALPHABET[i]] = i
        table.append([i, ALPHABET[i]])
    
    # w <- first char of input
    w = T[0]
    T = T[1:]

    # repeat
    while (T):
        # k <- next char
        k = T[0]
        T = T[1:]

        # if (EOF)
        if k == EOF:
            # output code(w)
            E.append(D[w])
        
        # else if (w * k) in Dictionary
        elif w+k in D:
            # w <- w * k # (remembe: '*' is concatenation, not mul)
            w = w + k
        
        # else
        else:
            # output code(w)
            E.append(D[w])
            # Dictionary <- w * k
            D[w+k] = len(D)
            table.append([len(D), w+k])
            # w <- k
            w = k
    
    return E, tabulate(table, headers=["code", "phrase"], tablefmt="pretty")

In [6]:
def lzw_decode(E, ALPHABET):
    """
    lzw decoder
    Input: list of codes E and relevant ALPHABET list
    Output: decoded text T and table represent the dictionary (tabulate)
    """
    ALPHABET = sorted(ALPHABET) # sort for strinct order..
    D        = {} # dictionary {code:phrase}
    T        = [] # the decoded text as phrases
    table    = [] # presenting the full dictionary D as table (tabulate type)
    
    # Initialize table with single character strings
    for i in range(len(ALPHABET)):
        D[i] = ALPHABET[i]
        table.append([i, ALPHABET[i]])
    
    # OLD = fisrt input code
    OLD = E[0]
    E = E[1:]

    # output translation of OLD
    T.append(D[OLD])

    # while not end of input stream
    C = ''
    while (E):
        # NEW = next input code
        NEW = E[0]
        E   = E[1:]

        # if NEW is not in the string table
        if NEW not in D:
            # S = translation of OLD
            S = D[OLD]
            # S = S * C # C is first character of S
            S = S + C
        
        # else
        else:
            # S = translation of NEW
            S = D[NEW]
        
        # output S
        T.append(S)
        # C = first character of S
        C = S[0]
        # Translation(OLD) * C to the string table
        D[len(D)] = D[OLD] + C
        table.append([len(D)-1, D[OLD] + C])
        # OLD = NEW
        OLD = NEW

    return T, tabulate(table, headers=["code", "phrase"], tablefmt="pretty")

In [7]:
# Example for PDF 8 slide 11:
T = "wabba_wabba_wabba_wabba_woo_woo"
ALPHABET = list(set(list(T)))
print("Text: {}, ALPHABET: {}".format(T, ALPHABET))

# encode
E, table = lzw_encode(T)
print(table)
print("encode({})={}".format(T, E))

# decode
T, table = lzw_decode(E, ALPHABET)
print(table)
print("decode({})={}".format(E, ''.join(T)))

Text: wabba_wabba_wabba_wabba_woo_woo, ALPHABET: ['w', 'b', 'o', 'a', '_']
+------+--------+
| code | phrase |
+------+--------+
|  0   |   _    |
|  1   |   a    |
|  2   |   b    |
|  3   |   o    |
|  4   |   w    |
|  6   |   wa   |
|  7   |   ab   |
|  8   |   bb   |
|  9   |   ba   |
|  10  |   a_   |
|  11  |   _w   |
|  12  |  wab   |
|  13  |  bba   |
|  14  |  a_w   |
|  15  |  wabb  |
|  16  |  ba_   |
|  17  |  _wa   |
|  18  |  abb   |
|  19  |  ba_w  |
|  20  |   wo   |
|  21  |   oo   |
|  22  |   o_   |
|  23  |  _wo   |
+------+--------+
encode(wabba_wabba_wabba_wabba_woo_woo)=[4, 1, 2, 2, 1, 0, 5, 7, 9, 11, 8, 10, 6, 15, 4, 3, 3, 10, 20]
+------+--------+
| code | phrase |
+------+--------+
|  0   |   _    |
|  1   |   a    |
|  2   |   b    |
|  3   |   o    |
|  4   |   w    |
|  5   |   wa   |
|  6   |   ab   |
|  7   |   bb   |
|  8   |   ba   |
|  9   |   a_   |
|  10  |   _w   |
|  11  |  wab   |
|  12  |  bba   |
|  13  |  a_w   |
|  14  |  wabb  |
|  15  |  ba

In [8]:
# Example for PDF 8 slide 13:
E = [0,1,3,2,4,7,0,9,10,0]
ALPHABET = ['a', 'b', 'c']

# docode
T, table = lzw_decode(E, ALPHABET)
print(table)
print("decode({})={}".format(E, ' '.join(T)))

+------+--------+
| code | phrase |
+------+--------+
|  0   |   a    |
|  1   |   b    |
|  2   |   c    |
|  3   |   ab   |
|  4   |   ba   |
|  5   |  abc   |
|  6   |   cb   |
|  7   |  bab   |
|  8   |  baba  |
|  9   |   aa   |
|  10  |  aaa   |
|  11  |  aaaa  |
+------+--------+
decode([0, 1, 3, 2, 4, 7, 0, 9, 10, 0])=a b ab c ba bab a aa aaa a
